In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv')
test=pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv')

In [ ]:
print('Train shape', train.shape)
train.head(3)

In [ ]:
print('Test shape', test.shape)
test.head(3)

In [ ]:
import os
import polars as pl
from scipy.optimize import minimize, Bounds
import kaggle_evaluation.default_inference_server

MAX_POSITION = 2.0
MIN_POSITION = 0.0
ANNUAL_DAYS = 252
EPS = 1e-12  

class UserVisibleError(Exception):
    
    pass

def adjusted_sharpe(solution: pd.DataFrame, submission: pd.DataFrame) -> float:

    solution = solution.copy()
    solution['position'] = submission['prediction'].astype(float)

    if solution['position'].max() > MAX_POSITION + 1e-9:
        raise UserVisibleError(f"Prediction above max limit {MAX_POSITION}")
    if solution['position'].min() < MIN_POSITION - 1e-9:
        raise UserVisibleError(f"Prediction below min limit {MIN_POSITION}")

    
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )

    excess = solution['strategy_returns'] - solution['risk_free_rate']

    
    cum_excess = float((1.0 + excess).prod())
    
    if cum_excess <= 0:
        mean_excess = -1.0 
    else:
        mean_excess = cum_excess ** (1.0 / len(solution)) - 1.0

    std_excess = float(solution['strategy_returns'].std())
    if std_excess == 0:
        
        raise ZeroDivisionError("Zero strategy std; Sharpe undefined.")

    sharpe = mean_excess / (std_excess + EPS) * np.sqrt(ANNUAL_DAYS)
    strat_vol = float(std_excess * np.sqrt(ANNUAL_DAYS) * 100.0)

    
    market_excess = solution['forward_returns'] - solution['risk_free_rate']
    market_cum = float((1.0 + market_excess).prod())
    if market_cum <= 0:
        market_mean = -1.0
    else:
        market_mean = market_cum ** (1.0 / len(solution)) - 1.0

    market_std = float(solution['forward_returns'].std())
    market_vol = float(market_std * np.sqrt(ANNUAL_DAYS) * 100.0)

    
    excess_vol_penalty = (1.0 + max(0.0, strat_vol / (market_vol + EPS) - 1.2)) if market_vol > 0 else 1.0
    return_gap = max(0.0, (market_mean - mean_excess) * 100.0 * ANNUAL_DAYS)
    return_penalty = 1.0 + (return_gap ** 2) / 100.0

    score = sharpe / (excess_vol_penalty * return_penalty + EPS)
    return float(min(score, 1_000_000.0))


train = pd.read_csv(
    "/kaggle/input/hull-tactical-market-prediction/train.csv",
    index_col="date_id"
)

for col in ["forward_returns", "risk_free_rate"]:
    if col not in train.columns:
        raise RuntimeError(f"Missing column in train: {col}")

In [ ]:
N_DAYS = 180
if len(train) < N_DAYS:
    N_DAYS = len(train)

recent = train.iloc[-N_DAYS:].copy()

def objective(x):
    
    positions = np.clip(np.asarray(x, dtype=float), MIN_POSITION, MAX_POSITION)
    submission = pd.DataFrame({'prediction': positions}, index=recent.index)
    
    return -adjusted_sharpe(recent, submission)

x0 = np.full(N_DAYS, 0.05, dtype=float)
bounds = Bounds(MIN_POSITION, MAX_POSITION)

res = minimize(
    objective,
    x0,
    method="Powell",
    bounds=bounds,
    tol=1e-8,
    options={"maxiter": 2000, "xtol": 1e-8, "ftol": 1e-8, "disp": True}
)
print("Optimization result:", res)

optimal_preds = np.clip(res.x if res.success else x0, MIN_POSITION, MAX_POSITION).astype(np.float64)

counter = 0  

def predict(batch: pl.DataFrame) -> float:

    global counter, optimal_preds
    if counter < len(optimal_preds):
        value = optimal_preds[counter]
    else:
        value = optimal_preds[-1]  
    print(f"[{counter}] Prediction: {float(value):.8f}")
    counter += 1
    
    return float(value)


In [ ]:
server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    server.serve()
else:
    server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))